In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Import Graphs

In [2]:
import os
import networkx as nx

datasets = []

random_graph_directory = './graphs/'
for root, dirs, files in os.walk(random_graph_directory):
    for filename in files:
        name_string = filename[:-4]
        graph_file_path = os.path.join(root, filename)

        G = nx.read_gml(graph_file_path)

        datasets.append(
            {
                "name": name_string,
                "graph": nx.relabel.convert_node_labels_to_integers(
                    G, first_label=0
                ),
            }
        )

## Define Solver Parameters

In [3]:
from solvers.dNNMIS_GPU_TAU import DNNMIS as V2
from solvers.KaMIS import ReduMIS
from solvers.ILPMIS import ILPMIS

solvers = [
    {
        "name": "dNN V2 GPU",
        "class": V2,
        "params": {
            "learning_rate": 0.001,
            "selection_criteria": 0.65,
            "max_steps": 100000,
        },
    },
    # {
    #     "name": "dNN",
    #     "class": DNNMIS,
    #     "params": {
    #         "learning_rate": 0.001,
    #         "selection_criteria": 0.8,
    #         "max_steps": 25000,
    #     },
    # },
    #     {
    #     "name": "dNN w/SG5k",
    #     "class": DNNMIS,
    #     "params": {
    #         "learning_rate": 0.001,
    #         "selection_criteria": 0.65,
    #         "max_steps": 80000,
    #         "max_subgraph_steps": 5000,
    #     },
    # },
    {"name": "ReduMIS", "class": ReduMIS, "params": {"seed": 13}},
    # {"name": "ILP", "class": ILPMIS, "params": {"time_limit": 935}}
]

## Optional: Import Existing Benchmark

In [4]:
import pickle

def read_object(filename):
    with open(filename, 'rb') as outp:
        return pickle.load(outp)
    
solutions.extend(read_object("solutions.pkl"))

NameError: name 'solutions' is not defined

## Optional: Run New Benchmark

In [5]:
from copy import deepcopy

def benchmark(datasets, solvers):
    solutions = []

    stage = 0
    stages = len(solvers) * len(datasets)

    for solver in solvers:
        for dataset in datasets[8:9]:
            solver_instance = solver["class"](dataset["graph"], solver["params"])
            solver_instance.solve()
            solution = {
                "solution_method": solver["name"],
                "dataset_name": dataset["name"],
                "data": deepcopy(solver_instance.solution),
                "time_taken": deepcopy(solver_instance.solution_time),
            }
            solutions.append(solution)
            del solver_instance
            stage += 1
            print(f"Completed {stage} / {stages}")

    return solutions

solutions = benchmark(datasets, solvers)

using device:  cuda:0
[0.22322221100330353, 0.3898888826370239, 0.13988889753818512, 0.36211109161376953, 0.36211109161376953, 0.25099998712539673, 0.5009999871253967, 0.22322221100330353, 0.3065555691719055, 0.19544441998004913, 0.5009999871253967, 0.4176666736602783, 0.36211109161376953, 0.36211109161376953, 0.36211109161376953, 0.4176666736602783, 0.3065555691719055, 0.36211109161376953, 0.3065555691719055, 0.25099998712539673, 0.36211109161376953, 0.2787777781486511, 0.22322221100330353, 0.47322219610214233, 0.25099998712539673, 0.3898888826370239, 0.33433330059051514, 0.25099998712539673, 0.25099998712539673, 0.36211109161376953, 0.4176666736602783, 0.2787777781486511, 0.44544440507888794, 0.19544441998004913, 0.3898888826370239, 0.3065555691719055, 0.25099998712539673, 0.36211109161376953, 0.25099998712539673, 0.4176666736602783, 0.0, 0.4176666736602783, 0.36211109161376953, 0.2787777781486511, 0.25099998712539673, 0.13988889753818512, 0.36211109161376953, 0.3065555691719055, 0.3

## Generate Results Table

In [ ]:
import pandas
import matplotlib.pyplot as plt

dataset_index = {k: v for v, k in enumerate([dataset["name"] for dataset in datasets])}
datasets_solutions = [[] for i in range(len(datasets))]

table_data = []

for solution in solutions:
    dsi = dataset_index[solution["dataset_name"]]
    datasets_solutions[dsi].append(solution)

i = 0
for dataset_solutions in datasets_solutions[8:9]:
    # IS CHECK
    is_check = []
    for solution in dataset_solutions:
        IS_set = solution["data"]["graph_mask"]
        subgraph = datasets[dataset_index[solution["dataset_name"]]]["graph"].subgraph(
            IS_set
        )
        # if len(subgraph.edges) > 0:
        #     plt.figure(i)
        #     plt.title(subgraph.edges)
        #     i += 1
        #     nx.draw(datasets[dataset_index[solution["dataset_name"]]]["graph"], with_labels=True, node_color=IS_set)
        #     is_check.append(False)
        #     print(
        #         f"Non IS found using {solution['solution_method']} on {solution['dataset_name']}"
        #     )
        # else:
        is_check.append(True)

    table_row = [dataset_solutions[0]['dataset_name']]

    table_row.extend([solution["data"]["size"] for solution in dataset_solutions])
    table_row.extend([solution["time_taken"] for solution in dataset_solutions])
    table_row.extend(is_check)

    table_data.append(table_row)

table_headers = ["Dataset Name"]

table_headers.extend([solver["name"] + " Solution Size" for solver in solvers])
table_headers.extend([solver["name"] + " Solution Time" for solver in solvers])
table_headers.extend([solver["name"] + " Solution IS" for solver in solvers])

table = pandas.DataFrame(table_data, columns=table_headers)
table
table.to_csv("results.csv")

## Optional: Find Exact Sets
```{note}
Result is indexed by 0
```

In [ ]:
graph_of_interest = "GNM_n50_m613_seed6"

for dataset_solutions in datasets_solutions:
    for solution in dataset_solutions:
        if (solution["dataset_name"] == graph_of_interest):
            set_solution = []
            for idx, x in enumerate(solution["data"]["graph_mask"]):
                if x == 1:
                    set_solution.append(idx)
            print(solution["solution_method"],set_solution)

## Optional: Save Solutions to Pickle

In [ ]:
import pickle

def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

def read_object(filename):
    with open(filename, 'rb') as outp:  # Overwrites any existing file.
        return pickle.load(outp)

solutions = read_object("solutions.pkl")